### multi input model

In [7]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(
    text_vocabulary_size, 64)(text_input)

encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,),
                       dtype='int32',
                       name='question')

embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question],
                                  axis=-1)

answer = layers.Dense(answer_vocabulary_size,
                      activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])




In [8]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))
answers = np.random.randint(0, 1,
                            size=(num_samples, answer_vocabulary_size))

model.fit([text, question], answers, epochs=10, batch_size=128)

# model.fit({'text': text, 'question': question}, answers,
#           epochs=10, batch_size=128)




Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 10/10
1000/1000 [==============================] 

### multi output model

In [1]:
from keras import layers
from keras import Input
from keras.models import Model
vocabulary_size = 500
num_income_groups = 10
num_age_range = 100

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size,256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
# income_prediction = layers.Dense(num_income_groups,
income_prediction = layers.Dense(10,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])



Using TensorFlow backend.


In [ ]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})


In [2]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})


In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    128000      posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    163968      embedding_1[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (

In [5]:
import numpy as np
from keras.utils import to_categorical

vocabulary_size = 500
num_income_groups = 10
num_age_range = 100

num_samples = 1000
max_length = 1000

posts = np.random.randint(1, vocabulary_size,
                         size=(num_samples, max_length))
income_target = np.random.randint(1, num_income_groups,
                             size=(num_samples, 1))
income_targets = to_categorical(income_target)

gender_targets = np.random.randint(0, 2,
                            size=(num_samples, 1))

 
age_targets = np.random.randint(1, num_age_range,
                             size=(num_samples, 1))


model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

# model.fit(posts, {'age': age_targets,
#                   'income': income_targets,
#                   'gender': gender_targets},
#           epochs=10, batch_size=64)


Epoch 1/10
1000/1000 [==============================] - 29s 29ms/step - loss: 7163.7277 - age_loss: 28568.5789 - income_loss: 6.4441 - gender_loss: 1.5139
Epoch 2/10
1000/1000 [==============================] - 28s 28ms/step - loss: 222.5397 - age_loss: 840.8444 - income_loss: 3.9234 - gender_loss: 0.8405
Epoch 3/10
1000/1000 [==============================] - 28s 28ms/step - loss: 223.0368 - age_loss: 844.7840 - income_loss: 3.6481 - gender_loss: 0.8193
Epoch 4/10
1000/1000 [==============================] - 28s 28ms/step - loss: 237.5856 - age_loss: 899.4771 - income_loss: 3.7206 - gender_loss: 0.8996
Epoch 5/10
1000/1000 [==============================] - 28s 28ms/step - loss: 366.2533 - age_loss: 1367.8870 - income_loss: 5.1797 - gender_loss: 1.9102
Epoch 6/10
1000/1000 [==============================] - 28s 28ms/step - loss: 388.2616 - age_loss: 1264.8313 - income_loss: 9.0603 - gender_loss: 6.2993
Epoch 7/10
1000/1000 [==============================] - 29s 29ms/step - loss: 397.1

### directed acyclic graph

#### inception e.g.

In [ ]:
from keras import layers
branch_a = layers.Conv2D(128, 1,
                         activation='relu', strides=2)(x)
branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)
output = layers.concatenate(
    [branch_a, branch_b, branch_c, branch_d], axis=-1)
	

#### residual connection (identity residual connection)

In [ ]:
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x])


#### residual connection (linear residual connection)

In [ ]:
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])

#### siamese LSTM (sharing weight)

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)


#### sharing same weight (左右攝影鏡頭使用同一模型訓練) 

In [ ]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None,
                                      include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_input = xception_base(right_input)

merged_features = layers.concatenate(
    [left_features, right_input], axis=-1)
	

#### callback EarlyStopping & ModelCheckpoint

In [ ]:
import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',
        patience=1,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5',
        monitor='val_loss',
        save_best_only=True,
    )
]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))
		  

#### callback ReduceLROnPlateau

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss'
        factor=0.1,
        patience=10,
    )
]

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))
		  

#### callback自定

In [ ]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):

    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input,
                                                    layer_outputs)

    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')

        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()
		

## TensorBoard e.g. IMDB

In [2]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [9]:
!mkdir my_log_dir

In [10]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',
        histogram_freq=1,
        embeddings_freq=1,
    )
]
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)	


Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 67s 3ms/step - loss: 0.6290 - acc: 0.6561 - val_loss: 0.5201 - val_acc: 0.7878
Epoch 2/20
20000/20000 [==============================] - 69s 3ms/step - loss: 0.4393 - acc: 0.8139 - val_loss: 0.4302 - val_acc: 0.8306
Epoch 3/20
20000/20000 [==============================] - 72s 4ms/step - loss: 0.4035 - acc: 0.8206 - val_loss: 0.4360 - val_acc: 0.8272
Epoch 4/20
20000/20000 [==============================] - 68s 3ms/step - loss: 0.3476 - acc: 0.7926 - val_loss: 0.5646 - val_acc: 0.7546
Epoch 5/20
20000/20000 [==============================] - 73s 4ms/step - loss: 0.3002 - acc: 0.7572 - val_loss: 0.5078 - val_acc: 0.7352
Epoch 6/20
20000/20000 [==============================] - 71s 4ms/step - loss: 0.2715 - acc: 0.7092 - val_loss: 0.5683 - val_acc: 0.6668
Epoch 7/20
20000/20000 [==============================] - 76s 4ms/step - loss: 0.2376 - acc: 0.6494 - val_loss: 0.7154 - val_acc:

#### tensorboard --logdir=my_log_dir
#### 開http://localhost:6006

## keras.utils.plot_model

In [3]:
# https://blog.csdn.net/linxid/article/details/79386223
# install pydot 
# install pydot-ng
# install graphviz
# 找到graphviz的bin/graphviz路徑添加到環境變量

from keras.utils import plot_model
plot_model(model, to_file='model.png')


In [4]:
from keras.utils import plot_model

plot_model(model, show_shapes=True, to_file='model2.png')

### BatchNormalization

In [ ]:
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

dense_model.add(layers.Dense(32, activation='relu'))
dense_model.add(layers.BatchNormalization())		

### depthwise separable convolution

In [19]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3,
                                 activation='relu',
                                 input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [13]:
from keras.utils import plot_model

plot_model(model, show_shapes=True, to_file='test.png')

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_38 (Separab (None, 62, 62, 32)        155       
_________________________________________________________________
separable_conv2d_39 (Separab (None, 60, 60, 64)        2400      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
separable_conv2d_40 (Separab (None, 28, 28, 64)        4736      
_________________________________________________________________
separable_conv2d_41 (Separab (None, 26, 26, 128)       8896      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 13, 13, 128)       0         
_________________________________________________________________
separable_conv2d_42 (Separab (None, 11, 11, 128)       17664     
__________

### hyperparameter

In [21]:
!pip install hyperas
# !python -m pip install --upgrade pip
# example: simple_notebook (hyperas example).ipynb
# example: simple_notebook_multiple_inputs (hyperas example).ipynb

  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1
